## Work
1. 請嘗試將 preproc_x 替換成以每筆資料的 min/max 進行標準化至 -1 ~ 1 間，再進行訓練
2. 請嘗試將 mlp 疊更深 (e.g 5~10 層)，進行訓練後觀察 learning curve 的走勢
3. (optional) 請改用 GPU 進行訓練 (如果你有 GPU 的話)，比較使用 CPU 與 GPU 的訓練速度

In [1]:
## Your code here (optional)
!nvidia-smi

'nvidia-smi' 不是內部或外部命令、可執行的程式或批次檔。


In [2]:
import os
import keras

# 請嘗試設定 GPU：os.environ
os.environ["CUDA_VISIBLE_DEVICES"] = ""

Using TensorFlow backend.


In [3]:
train, test = keras.datasets.cifar10.load_data()

In [4]:
from sklearn import preprocessing
## 資料前處理

def preproc_x(x, flatten=True):
     
    # 變成一維
    if flatten:
        x = x.reshape((len(x), -1))
        min_max_scaler = preprocessing.MinMaxScaler()
        x = min_max_scaler.fit_transform(x)
    return x
  
def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y

In [5]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

C:\Users\blues\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype uint8 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\blues\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype uint8 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [6]:
# 建立神經網路
def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128, 64, 32]):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, activation="relu", name="hidden_layer"+str(i+1))(input_layer)
        else:
            x = keras.layers.Dense(units=n_units, activation="relu", name="hidden_layer"+str(i+1))(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model
  
model = build_mlp(input_shape=x_train.shape[1:])
model.summary()

W0731 20:10:11.502155 15720 deprecation_wrapper.py:119] From C:\Users\blues\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0731 20:10:12.251707 15720 deprecation_wrapper.py:119] From C:\Users\blues\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0731 20:10:12.562691 15720 deprecation_wrapper.py:119] From C:\Users\blues\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
hidden_layer4 (Dense)        (None, 64)                8256      
_________________________________________________________________
hidden_layer5 (Dense)        (None, 32)                2080      
_________________________________________________________________
output (Dense)               (None, 10)                330       
Total para

In [7]:
## 超參數設定
LEARNING_RATE = 0.001
EPOCHS = 100
BATCH_SIZE = 256

In [8]:
optimizer = keras.optimizers.Adam(lr=LEARNING_RATE)
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

W0731 20:10:13.419150 15720 deprecation_wrapper.py:119] From C:\Users\blues\Anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0731 20:10:13.454303 15720 deprecation_wrapper.py:119] From C:\Users\blues\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [9]:
model.fit(x_train, y_train, 
          epochs=EPOCHS, 
          batch_size=BATCH_SIZE, 
          validation_data=(x_test, y_test), 
          shuffle=True)

W0731 20:10:13.784683 15720 deprecation.py:323] From C:\Users\blues\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0731 20:10:14.096253 15720 deprecation_wrapper.py:119] From C:\Users\blues\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - 24s 475us/step - loss: 2.0009 - acc: 0.2710 - val_loss: 1.8528 - val_acc: 0.3402
Epoch 2/100
50000/50000 [==============================] - 21s 418us/step - loss: 1.7630 - acc: 0.3670 - val_loss: 1.6978 - val_acc: 0.3913
Epoch 3/100
50000/50000 [==============================] - 23s 453us/step - loss: 1.6616 - acc: 0.4038 - val_loss: 1.6256 - val_acc: 0.4184
Epoch 4/100
50000/50000 [==============================] - 24s 473us/step - loss: 1.5877 - acc: 0.4344 - val_loss: 1.5972 - val_acc: 0.4301
Epoch 5/100
50000/50000 [==============================] - 20s 399us/step - loss: 1.5472 - acc: 0.4480 - val_loss: 1.5915 - val_acc: 0.4402
Epoch 6/100
50000/50000 [==============================] - 21s 424us/step - loss: 1.5045 - acc: 0.4629 - val_loss: 1.5151 - val_acc: 0.4574
Epoch 7/100
50000/50000 [==============================] - 21s 426us/step - loss: 1.4736 - acc: 0.4732 - val_l

Epoch 59/100
50000/50000 [==============================] - 20s 407us/step - loss: 0.5730 - acc: 0.7908 - val_loss: 1.9846 - val_acc: 0.5170
Epoch 60/100
50000/50000 [==============================] - 20s 404us/step - loss: 0.5487 - acc: 0.8029 - val_loss: 1.9978 - val_acc: 0.5263
Epoch 61/100
50000/50000 [==============================] - 21s 421us/step - loss: 0.5447 - acc: 0.8041 - val_loss: 2.0625 - val_acc: 0.5168
Epoch 62/100
50000/50000 [==============================] - 21s 410us/step - loss: 0.5393 - acc: 0.8065 - val_loss: 2.0963 - val_acc: 0.5110
Epoch 63/100
50000/50000 [==============================] - 20s 403us/step - loss: 0.5088 - acc: 0.8167 - val_loss: 2.1143 - val_acc: 0.5164
Epoch 64/100
50000/50000 [==============================] - 20s 400us/step - loss: 0.5167 - acc: 0.8122 - val_loss: 2.1407 - val_acc: 0.5120
Epoch 65/100
50000/50000 [==============================] - 20s 405us/step - loss: 0.4981 - acc: 0.8211 - val_loss: 2.1955 - val_acc: 0.5160
Epoch 66/100


In [10]:
import matplotlib.pyplot as plt


train_loss = model.history.history["loss"]
valid_loss = model.history.history["val_loss"]

train_acc = model.history.history["acc"]
valid_acc = model.history.history["val_acc"]

plt.plot(range(len(train_loss)), train_loss, label="train loss")
plt.plot(range(len(valid_loss)), valid_loss, label="valid loss")
plt.legend()
plt.title("Loss")
plt.show()

plt.plot(range(len(train_acc)), train_acc, label="train accuracy")
plt.plot(range(len(valid_acc)), valid_acc, label="valid accuracy")
plt.legend()
plt.title("Accuracy")
plt.show()

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>